<a href="https://colab.research.google.com/github/hpn-nina/simple-image-application/blob/master/Palette_Inpainting_CelebaHQ_256.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Image inpainting
A demo for image inpainting on Celeba-HQ based on [Palette](https://github.com/Janspiry/Palette-Image-to-Image-Diffusion-Models).

## Setup

### Get the GPU

*   Turn on hardware acceleration under `Runtime -> Change Runtime Type -> Hardware accelerator -> GPU`
*   Use this command to ensure that the connected machine has a GPU:

In [ ]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla T4, 460.32.03, 15109 MiB


### Clone repo

In [ ]:
%cd /content/
!git clone https://github.com/Janspiry/Palette-Image-to-Image-Diffusion-Models

/content
Cloning into 'Palette-Image-to-Image-Diffusion-Models'...
remote: Enumerating objects: 305, done.
remote: Counting objects: 100% (305/305), done.
remote: Compressing objects: 100% (211/211), done.
remote: Total 305 (delta 169), reused 224 (delta 90), pack-reused 0
Receiving objects: 100% (305/305), 6.87 MiB | 22.54 MiB/s, done.
Resolving deltas: 100% (169/169), done.


### Install requirements

In [ ]:
!pip list
# The default environment can working properly

Package                       Version
----------------------------- ------------------------------
absl-py                       1.1.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.2.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.4.0
audioread                     2.1.9
autograd                      1.4
Babel                         2.10.2
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.0
blis                          0.7.7
bokeh                         2.3.3
branca                        0.5.0
bs4                           0.0.1
CacheControl                  0.

### Prepare pertrained-model
Two ways to prepare your model are provided below.



#### Download from the google driver
1. Go [here](https://drive.google.com/drive/folders/13YZ2UAmGJ-b7DICr-FDAPM7gctreJEoH?usp=sharing), right click on `.pth` file, and hit `Make a copy`. Next, locate the files in your own google drive, and rename them from `Copy of 200_Network.pth` to `200_Network.pth`. Note, `.state` file is not required for the simple inference.
2. Next, right click on each file, and hit `Get link`. Click on `Restricted`, and change it to `Anyone with the link...`, as `Viewer`. Now, copy the links. They should look something like this: `https://drive.google.com/file/d/<file id>/view?usp=sharing`. Copy the file id, and paste them into the `gdown` commands below. 


In [ ]:
%cd /content/Palette-Image-to-Image-Diffusion-Models/
!gdown --id 1gzNs8_9LuysjbWsHftXdHp0jDYOyIgY1

/content/Palette-Image-to-Image-Diffusion-Models
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1gzNs8_9LuysjbWsHftXdHp0jDYOyIgY1
To: /content/Palette-Image-to-Image-Diffusion-Models/200_Network.pth
100% 251M/251M [00:01<00:00, 178MB/s]


#### Upload from the localhost
If `gdown` command did not work, you can manually upload files for the pre-trained model.
1. Download the [files](https://drive.google.com/drive/folders/13YZ2UAmGJ-b7DICr-FDAPM7gctreJEoH?usp=sharing) to localhost. 
2. Click the `files` button on the left hand side of the screen, and find the folder labled `Palette-Image-to-Image-Diffusion-Models`. Click the three dots on the right of the folder, and click `upload`. Then, select the model files that you downloaded.

### Edit configure file

In [ ]:
%cd /content/Palette-Image-to-Image-Diffusion-Models/config/

/content/Palette-Image-to-Image-Diffusion-Models/config


#### Create patchfile

Create patchfile, it will update the configuration file to inference for centering mask.

In [ ]:
%%writefile inpainting_celebahq.patch
--- a/config/inpainting_celebahq.json
+++ b/config/inpainting_celebahq.json
@@ -10,7 +10,7 @@
         "tb_logger": "tb_logger", // path of tensorboard logger
         "results": "results",
         "checkpoint": "checkpoint",
-        "resume_state": "experiments/train_inpainting_celebahq_220426_233652/checkpoint/190" 
+        "resume_state": "200" 
         // "resume_state": null // ex: 100, loading .state  and .pth from given epoch and iteration
     },
 
@@ -48,7 +48,7 @@
             "which_dataset": {
                 "name": "InpaintDataset", // import Dataset() class / function(not recommend) from default file
                 "args":{
-                    "data_root": "datasets/celebahq/flist/test.flist",
+                    "data_root": "input",
                     "mask_config": {
                         "mask_mode": "center"
                     }
@@ -56,8 +56,8 @@
             },
             "dataloader":{
                 "args":{
-                    "batch_size": 8,
-                    "num_workers": 4,
+                    "batch_size": 1,
+                    "num_workers": 1,
                     "pin_memory": true
                 }
             }
--

Writing inpainting_celebahq.patch


#### Apply the patch file
The error `patch unexpectedly ends in middle of line` can be ignored.

In [ ]:
!apt-get install dos2unix
!dos2unix inpainting_celebahq.json
!patch < inpainting_celebahq.patch

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  dos2unix
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 351 kB of archives.
After this operation, 1,267 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 dos2unix amd64 7.3.4-3 [351 kB]
Fetched 351 kB in 1s (368 kB/s)
Selecting previously unselected package dos2unix.
(Reading database ... 155639 files and directories currently installed.)
Preparing to unpack .../dos2unix_7.3.4-3_amd64.deb ...
Unpacking dos2unix (7.3.4-3) ...
Setting up dos2unix (7.3.4-3) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
dos2unix: converting file inpainting_celebahq.json to Unix format...
patching file inpainting_celebahq.json
patch unexpect

### Upload test data
Upload the 256*256 face images.

In [ ]:
%cd /content/Palette-Image-to-Image-Diffusion-Models/
!mkdir -p input
%cd /content/Palette-Image-to-Image-Diffusion-Models/input/
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  savefile = open(fn, 'wb')
  savefile.write(uploaded[fn])
  print('Successfully uploaded "{}" ({} bytes).'.format(fn, len(uploaded[fn])))
  savefile.close()

/content/Palette-Image-to-Image-Diffusion-Models
/content/Palette-Image-to-Image-Diffusion-Models/input


Saving 00023.jpg to 00023.jpg
Successfully uploaded "00023.jpg" (9325 bytes).


## Inference
It will cost several minutes depending on allocated GPU. 

Output images will be located under ./experiments/\<some folder\>/results/test.

In [ ]:
%cd /content/Palette-Image-to-Image-Diffusion-Models/
!python run.py -c config/inpainting_celebahq.json -p test

/content/Palette-Image-to-Image-Diffusion-Models
export CUDA_VISIBLE_DEVICES=0
run.py:28: UserWarning: You have chosen to use cudnn for accleration. torch.backends.cudnn.enabled=True
  warnings.warn('You have chosen to use cudnn for accleration. torch.backends.cudnn.enabled=True')
  0% 0/1 [00:00<?, ?it/s]
sampling loop time step:   0% 0/1000 [00:00<?, ?it/s]
sampling loop time step:   0% 1/1000 [00:03<54:58,  3.30s/it]
sampling loop time step:   0% 3/1000 [00:03<15:22,  1.08it/s]
sampling loop time step:   0% 4/1000 [00:03<10:37,  1.56it/s]
sampling loop time step:   1% 6/1000 [00:03<06:08,  2.70it/s]
sampling loop time step:   1% 8/1000 [00:03<04:14,  3.89it/s]
sampling loop time step:   1% 10/1000 [00:04<03:14,  5.08it/s]
sampling loop time step:   1% 11/1000 [00:04<02:54,  5.65it/s]
sampling loop time step:   1% 13/1000 [00:04<02:24,  6.82it/s]
sampling loop time step:   2% 15/1000 [00:04<02:06,  7.76it/s]
sampling loop time step:   2% 17/1000 [00:04<01:55,  8.48it/s]
sampling loop